# Grouping

1. Simple grouping
2. More complex grouping
    - Grouping on more than one categorical column
    - Grouping on more than one numeric column
    - Using more than one aggregation method
3. Pivot tables (2D grouping -- on two categorical columns)
4. Stacking and unstacking -- moving things from rows to columns and back

In [2]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame

In [3]:
filename = '../data/taxi.csv'   # 10k taxi rides from NYC in 2015
df = pd.read_csv(filename)

In [4]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,2,2015-06-02 11:19:29,2015-06-02 11:47:52,1,1.63,-73.954430,40.764141,1,N,-73.974754,40.754093,2,17.0,0.0,0.5,0.00,0.0,0.3,17.80
1,2,2015-06-02 11:19:30,2015-06-02 11:27:56,1,0.46,-73.971443,40.758942,1,N,-73.978539,40.761909,1,6.5,0.0,0.5,1.00,0.0,0.3,8.30
2,2,2015-06-02 11:19:31,2015-06-02 11:30:30,1,0.87,-73.978111,40.738434,1,N,-73.990273,40.745438,1,8.0,0.0,0.5,2.20,0.0,0.3,11.00
3,2,2015-06-02 11:19:31,2015-06-02 11:39:02,1,2.13,-73.945892,40.773529,1,N,-73.971527,40.760330,1,13.5,0.0,0.5,2.86,0.0,0.3,17.16
4,1,2015-06-02 11:19:32,2015-06-02 11:32:49,1,1.40,-73.979088,40.776772,1,N,-73.982162,40.758999,2,9.5,0.0,0.5,0.00,0.0,0.3,10.30


In [6]:
# I want the mean trip_distance where passenger_count is 1

df.loc[
    df['passenger_count'] == 1     # row selector
    ,
    'trip_distance'    # column selector
].mean()

3.0923380047176354

In [7]:
# but what if I want to perform the same calculation for passenger_count == 2

df.loc[
    df['passenger_count'] == 2     # row selector
    ,
    'trip_distance'    # column selector
].mean()

3.3843869002284848

In [8]:
# but what if I want to perform the same calculation for passenger_count == 3

df.loc[
    df['passenger_count'] == 3     # row selector
    ,
    'trip_distance'    # column selector
].mean()

3.3423891625615765

What we're doing (manually) is taking each distinct/unique value in `passenger_count` and we're running our query on it.

The whole point of grouping is to ask Pandas to do this same task for us, calculating once for each distinct value in `passenger_count`.

The way to think about grouping is as follows:

- One categorical column, on which we'll do the grouping. We'll get one result for each distinct value in this column.
- One numeric column, on which we'll perform the calculation.
- One aggregation method, which will be invoked on all of the values in the numeric column for each distinct value of the categorical

In our example above:
- passenger_count is categorical
- trip_distance is numeric
- mean is an aggregation method

In [9]:
# df.groupby(CATEGORICAL)[NUMERIC].aggregate()

df.groupby('passenger_count')['trip_distance'].mean()

passenger_count
0    4.600000
1    3.092338
2    3.384387
3    3.342389
4    3.628901
5    3.182712
6    3.170976
Name: trip_distance, dtype: float64

In [10]:
# normally, it's not a bad thing that Pandas automatically sorts the distinct values in 
# passenger_count and then displays them in that order. However, if you're going to reorder
# them, or if you want to save a bit of calculation, you can pass the sort=False keyword
# argument.

df.groupby('passenger_count', sort=False)['trip_distance'].mean()

passenger_count
1    3.092338
4    3.628901
3    3.342389
2    3.384387
5    3.182712
6    3.170976
0    4.600000
Name: trip_distance, dtype: float64

In [11]:
# what does groupby return?

df.groupby('passenger_count')['trip_distance']

In [12]:
df.groupby('passenger_count', sort=False)['trip_distance'].mean().sort_index(ascending=False)

passenger_count
6    3.170976
5    3.182712
4    3.628901
3    3.342389
2    3.384387
1    3.092338
0    4.600000
Name: trip_distance, dtype: float64

In [13]:
filename = '../data/olympic_athlete_events.csv'   # all athletes from all Olympic games through 2020

!head $filename

"ID","Name","Sex","Age","Height","Weight","Team","NOC","Games","Year","Season","City","Sport","Event","Medal"
"1","A Dijiang","M",24,180,80,"China","CHN","1992 Summer",1992,"Summer","Barcelona","Basketball","Basketball Men's Basketball",NA
"2","A Lamusi","M",23,170,60,"China","CHN","2012 Summer",2012,"Summer","London","Judo","Judo Men's Extra-Lightweight",NA
"3","Gunnar Nielsen Aaby","M",24,NA,NA,"Denmark","DEN","1920 Summer",1920,"Summer","Antwerpen","Football","Football Men's Football",NA
"4","Edgar Lindenau Aabye","M",34,NA,NA,"Denmark/Sweden","DEN","1900 Summer",1900,"Summer","Paris","Tug-Of-War","Tug-Of-War Men's Tug-Of-War","Gold"
"5","Christine Jacoba Aaftink","F",21,185,82,"Netherlands","NED","1988 Winter",1988,"Winter","Calgary","Speed Skating","Speed Skating Women's 500 metres",NA
"5","Christine Jacoba Aaftink","F",21,185,82,"Netherlands","NED","1988 Winter",1988,"Winter","Calgary","Speed Skating","Speed Skating Women's 1,000 metres",NA
"5","Christine Jacoba Aaftink","F",25,1

# Exercise: Olympic calculations

1. Read the Olympic data into a data frame.
2. Find the mean age for people in each sport.
3. Find the max height for people in each sport after 1960.

In [14]:
df = pd.read_csv(filename)

In [15]:
df.shape

(271116, 15)

In [16]:
# find the mean age for people in each sport

df.groupby('Sport')['Age'].mean()

Sport
Aeronautics         26.000000
Alpine Skiing       23.205462
Alpinism            38.812500
Archery             27.935226
Art Competitions    45.901009
                      ...    
Tug-Of-War          29.309524
Volleyball          25.183800
Water Polo          25.659627
Weightlifting       25.502010
Wrestling           25.798289
Name: Age, Length: 66, dtype: float64

In [17]:
df.groupby('Sport')['Age'].mean().sort_values()

Sport
Rhythmic Gymnastics      18.737082
Swimming                 20.566803
Figure Skating           22.232190
Synchronized Swimming    22.366851
Diving                   22.481441
                           ...    
Equestrianism            34.390831
Polo                     35.333333
Alpinism                 38.812500
Art Competitions         45.901009
Roque                    53.333333
Name: Age, Length: 66, dtype: float64

In [22]:
# Find the max height for people in each sport after 1960.

(
    df
    .loc[df['Year'] > 1960]
    .groupby('Sport')['Height'].max()
)

Sport
Alpine Skiing                200.0
Archery                      197.0
Athletics                    208.0
Badminton                    201.0
Baseball                     206.0
Basketball                   226.0
Beach Volleyball             212.0
Biathlon                     200.0
Bobsleigh                    205.0
Boxing                       205.0
Canoeing                     205.0
Cross Country Skiing         200.0
Curling                      197.0
Cycling                      201.0
Diving                       188.0
Equestrianism                197.0
Fencing                      208.0
Figure Skating               193.0
Football                     200.0
Freestyle Skiing             197.0
Golf                         196.0
Gymnastics                   185.0
Handball                     214.0
Hockey                       200.0
Ice Hockey                   206.0
Judo                         213.0
Luge                         199.0
Modern Pentathlon            198.0
Nordic Combine

In [23]:
# can I groupby on a numeric column?
# yes - but be careful!

df = pd.read_csv('../data/taxi.csv')

# for every distinct value of trip_distance...
# get the mean total_amount
df.groupby('trip_distance')['total_amount'].mean()  

trip_distance
0.00      31.58194
0.01      52.80000
0.02      43.46000
0.03       3.96000
0.04      70.01000
           ...    
34.84    137.59000
35.51    135.13000
37.20    210.14000
60.30    160.05000
64.60     79.96000
Name: total_amount, Length: 1219, dtype: float64

# More complex grouping

What if we want (with the taxi data) to group not just by passenger_count, but also by vendorID (i.e., who made the taxi meter)?

Meaning: I want to know the mean `trip_distance` not just for every distinct value of `passenger_count`, but for every combination of `passenger_count` and `VendorID`. 

This requires that we remember a general rule of thumb in Pandas: Wherever you can pass/use a single column name, you can also pass a list of column names.